# Training Updated Style Models

This notebook trains four separate UNet-based autoencoder models for simple style imitation:

- **blur**
- **night_vis**
- **poster** (posterization style)
- **outline** (soft colored outline style)

Each model learns a mapping:
\begin{align}
\text{input RGB image} \;\rightarrow\; \text{styled RGB image}
\end{align}

The models are saved under `models/upd/autoencoder_{style}_upd.keras`.

In [1]:
# Imports & configuration
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# -------- PATHS --------
BASE_INPUT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input"
STYLE_ROOT = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style"
MODEL_SAVE_DIR = "/Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/models/upd"

IMAGE_SIZE = (256, 256)
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

# Style folders we expect under STYLE_ROOT
STYLES = ["blur", "night_vis", "poster", "outline"]
print("Planned styles:", STYLES)

Planned styles: ['blur', 'night_vis', 'poster', 'outline']


In [2]:
# Helper: load all RGB images from a folder and normalize to [0, 1]
def load_images_from_folder(folder, size=IMAGE_SIZE):
    imgs = []
    files = sorted([
        f for f in os.listdir(folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tiff"))
    ])

    if not files:
        print(f"[WARN] No image files found in: {folder}")

    for fname in files:
        path = os.path.join(folder, fname)
        img = cv2.imread(path)
        if img is None:
            print("[WARN] Skipping unreadable:", path)
            continue

        # BGR -> RGB for consistency
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, size)
        img = img.astype("float32") / 255.0
        imgs.append(img)

    return np.array(imgs)

In [3]:
# UNet-style autoencoder architecture for 256x256x3 images
def build_unet_autoencoder(input_shape=(256, 256, 3)):
    inp = Input(shape=input_shape)

    # Encoder
    c1 = Conv2D(32, (3,3), activation="relu", padding="same")(inp)
    p1 = MaxPooling2D((2,2))(c1)              # 256 -> 128

    c2 = Conv2D(64, (3,3), activation="relu", padding="same")(p1)
    p2 = MaxPooling2D((2,2))(c2)              # 128 -> 64

    c3 = Conv2D(128, (3,3), activation="relu", padding="same")(p2)
    p3 = MaxPooling2D((2,2))(c3)              # 64 -> 32

    # Bottleneck
    bn = Conv2D(256, (3,3), activation="relu", padding="same")(p3)

    # Decoder
    u3 = UpSampling2D((2,2))(bn)              # 32 -> 64
    u3 = Concatenate()([u3, c3])
    d3 = Conv2D(128, (3,3), activation="relu", padding="same")(u3)

    u2 = UpSampling2D((2,2))(d3)              # 64 -> 128
    u2 = Concatenate()([u2, c2])
    d2 = Conv2D(64, (3,3), activation="relu", padding="same")(u2)

    u1 = UpSampling2D((2,2))(d2)              # 128 -> 256
    u1 = Concatenate()([u1, c1])
    d1 = Conv2D(32, (3,3), activation="relu", padding="same")(u1)

    # Here's the change: I've fixated the colors so they won't shift, and I added more samples to train from now on. Like, 15 more imgs lmao

    out = Conv2D(3, (3,3), activation="sigmoid", padding="same")(d1)
    out = out + inp
    out = tf.clip_by_value(out, 0.0, 1.0)


    model = Model(inp, out)
    model.compile(optimizer=Adam(1e-4), loss="mae")
    return model

print("UNet autoencoder ready.")

UNet autoencoder ready.


In [4]:
# Load input images once (these are the 'content' images)
print("Loading INPUT images from:", BASE_INPUT)
input_images = load_images_from_folder(BASE_INPUT)
print("Input images shape:", input_images.shape)

if input_images.size == 0:
    raise RuntimeError("No input images loaded. Check BASE_INPUT path and files.")

Loading INPUT images from: /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/input
Input images shape: (5, 256, 256, 3)


In [5]:
# Helper function: train one style-specific model
def train_style_model(style_name,
                      base_inputs,
                      epochs=40,
                      batch_size=2,
                      val_split=0.15):
    """Train a UNet autoencoder to map input -> style images for a given style_name."""

    style_folder = os.path.join(STYLE_ROOT, style_name)
    print("\n====================================")
    print(f" TRAINING STYLE: {style_name.upper()}")
    print(" Style folder:", style_folder)
    print("====================================")

    if not os.path.isdir(style_folder):
        print(f"[ERROR] Style folder does not exist: {style_folder}")
        return

    style_images = load_images_from_folder(style_folder)
    print("Style images shape:", style_images.shape)

    if style_images.size == 0:
        print("[ERROR] No style images found, skipping style:", style_name)
        return

    if len(style_images) != len(base_inputs):
        print("[ERROR] COUNT MISMATCH for style: ", style_name)
        print("        #input_images =", len(base_inputs))
        print("        #style_images =", len(style_images))
        print("        Ensure 1-1 pairing between input and style images.")
        return

    X_train, X_val, y_train, y_val = train_test_split(
        base_inputs, style_images,
        test_size=val_split,
        random_state=42
    )

    model = build_unet_autoencoder()

    save_path = os.path.join(MODEL_SAVE_DIR, f"autoencoder_{style_name}_upd.keras")
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
        ModelCheckpoint(save_path, monitor="val_loss", save_best_only=True)
    ]

    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=1
    )

    print(f"\nSaved updated model for style '{style_name}' to:\n  {save_path}\n")
    return history

print("Training helper ready.")

Training helper ready.


In [6]:
# Train BLUR model
hist_blur = train_style_model("blur", input_images, epochs=40, batch_size=2)


 TRAINING STYLE: BLUR
 Style folder: /Users/amayakof/Desktop/2025_autumn/deep_learning/SIS/3/project/data/preprocessed/style/blur
Style images shape: (5, 256, 256, 3)


NameError: name 'tf' is not defined

In [ ]:
# Train NIGHT_VIS model
hist_night = train_style_model("night_vis", input_images, epochs=40, batch_size=2)

In [ ]:
# Train POSTER model
hist_poster = train_style_model("poster", input_images, epochs=40, batch_size=2)

In [ ]:
# Train OUTLINE model
hist_outline = train_style_model("outline", input_images, epochs=40, batch_size=2)